### 1.什么是ReAct Agent
ReAct Agent是一种目前非常流行的Agent框架，核心思想是让模型在解决问题时动态交替进行逻辑推理和环境交互，模仿人类的决策过程。
ReAct Agent的核心工作流程包括以下三个步骤：
1. Reasoning（推理）：LLM分析用户的输入，思考需要做什么，决定是否需要调用工具来解决问题。
2. Acting（行动）：如果需要调用工具，LLM会根据推理结果调用相应的工具，并将工具的输出作为环境反馈。
3. Observation（观察）：LLM会根据工具的输出，更新自己的内部状态，继续推理和行动，直到问题解决。
这三个步骤形成一个循环：推理 -> 行动 -> 观察 -> 推理 ...直到LLM认为已经收集到足够的信息，可以给出最终答案为止。
#### 为什么需要ReAct Agent
ReAct模式的出现解决了传统AI Agent的几个关键问题：
1. 能力边界问题：纯LLM虽然知识丰富，但无法获取实时信息、执行具体操作或访问外部系统。ReAct模式能够通过工具调用扩展LLM的能力边界。
2. 决策智能性：ReAct模式能够让Agent能够根据具体情境去智能判断是否需要调用工具，以及调用哪个工具。
3. 复杂问题处理：对于需要多步骤推理的复杂问题，ReAct的循环机制允许Agent进行多次推理-行动-观察，逐步收集信息并完善答案。
### 2.工具调用
工具调用，是一种让AI Agent能够使用外部工具的机制。
某种意义上，Tools跟Message、Template一样，本质上都是根据不同的目的，对向模型输入的信息进行标准化、规范化的方式。
在LangGraph中，工具调用的流程通常为：
- LLM分析用户输入，决定是否需要调用工具；
- 如果需要，LLM会生成工具调用请求（tool_calls）；
- 工具节点（ToolNode）执行工具调用；
- 工具执行结果返回给LLM；
- LLM根据工具结果生成最终回复；
### 3.子图
子图（Subgraph）是指在LangGraph中，将一个复杂的任务分解为多个子任务，每个子任务对应一个子图。
每个子图都是一个独立的Agent，负责完成一个具体的子任务。
子图之间可以通过条件边（Conditional Edge）进行连接，根据子任务的执行结果，动态选择下一个子任务。
子图本身是一个独立的Graph，但它又能作为一个节点（Node）嵌入到另一个Graph（父图）中。
子图的优势在于：
1. 任务分解：将复杂任务分解为多个简单任务，每个任务由一个子图负责，使任务管理更加清晰。
2. 并行执行：多个子图可以并行执行，提高任务处理效率。
3. 可维护性：每个子图都是一个独立的模块，易于维护和调试。
### 4.简易ReAct Agent的代码实现
1. 一个子图，负责处理工具调用循环；
2. 一个主图，负责管理整个对话循环；


In [4]:
from typing import Annotated, Sequence, TypedDict, List
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from datetime import datetime

from langgraph.graph.state import END, START, StateGraph

load_dotenv()

# 子图的状态
class SubAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

llm = init_chat_model(
    model="Qwen/Qwen3-8B",
    model_provider="openai",
    temperature=0.5,
)

# 创建工具
@tool
def get_current_time() -> str:
    """获取当前的日期和时间，返回格式化的时间字符串。当用户询问时间相关问题时使用此工具。"""
    now = datetime.now()
    # 格式化日期和时间
    date_str = now.strftime("%Y年%m月%d日")
    time_str = now.strftime("%H:%M:%S")
    # 获取星期几
    weekdays = ["星期一", "星期二", "星期三", "星期四", "星期五", "星期六", "星期日"]
    weekday_str = weekdays[now.weekday()]
    # 判断上午/下午
    hour = now.hour
    if hour < 12:
        period = "上午"
    elif hour < 18:
        period = "下午"
    else:
        period = "晚上"
    # 返回友好的时间描述
    return f"当前时间是：{date_str} {weekday_str} {period} {time_str}"

tools = [get_current_time]

Agent = llm.bind_tools(tools)

# Chat_Bot 节点
def Chat_bot(state: SubAgentState) -> SubAgentState:
    """这个节点将使用LLM对用户的输入进行反馈"""
    system_prompt = SystemMessage(content="你是一个助手，请根据用户输入选择合适的工具（如有）来回复。")
    messages = [system_prompt] + state["messages"]
    response = Agent.invoke(messages)
    if response.content:
        print(f"\nAI: {response.content}\n")
    return {"messages": [response]}

# Tools节点
tool_node = ToolNode(tools=tools)

# 条件路由函数
def should_continue(state: SubAgentState) -> str:
    """根据AI响应决定是否需要使用工具（如有）"""
    last_message = state["messages"][-1]
    if not last_message.tool_calls:
        return "END"
    else:
        return "tool_node"

# 构建子图
sub_graph = StateGraph(SubAgentState)
sub_graph.add_node("chat_bot", Chat_bot)
sub_graph.add_node("tool_node", tool_node)

sub_graph.add_edge(START, "chat_bot")
sub_graph.add_conditional_edges(
    "chat_bot",
    should_continue,
    {
        "END": END,
        "tool_node": "tool_node",
    }
)
sub_graph.add_edge("tool_node", "chat_bot")
subgraph = sub_graph.compile()

# 构建主图
class AgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

def get_user_input(state: AgentState) -> AgentState:
    """获取用户输入"""
    user_input = input("请输入：")
    return {"messages": [HumanMessage(content=user_input)]}

def should_continue_main(state: AgentState) -> str:
    """根据用户的输入判断是否继续对话"""
    last_message = state["messages"][-1]
    if isinstance(last_message, HumanMessage) and last_message.content == "结束":
        return "END"
    else:
        return "subgraph"
# 构建主图
graph = StateGraph(AgentState)
# 添加节点
graph.add_node("get_user_input", get_user_input)
graph.add_node("subgraph", subgraph)
# 设置边
graph.add_edge(START, "get_user_input")
graph.add_conditional_edges(
    "get_user_input",
    should_continue_main,
    {
        "END": END,
        "subgraph": "subgraph",
    }
)
graph.add_edge("subgraph", "get_user_input")

app = graph.compile()
app.invoke({"messages": []})






AI: 你好！有什么可以帮助你的吗？


AI: 我是Qwen，是阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答各种问题、创作文字，比如写故事、写邮件、写剧本，甚至写代码。我还能进行多轮对话，保持上下文连贯，帮助用户解决问题。有什么具体的事情需要我的帮助吗？


AI: 现在是2026年02月26日 星期四 下午16:06:26。需要我帮你做些什么吗？


AI: 我目前可以调用的工具包括：

1. `get_current_time`：获取当前的日期和时间，返回格式化的时间字符串。适用于回答时间相关问题。

如果你有其他需求，可以告诉我，我会尽力为你提供帮助！



{'messages': [HumanMessage(content='你好', additional_kwargs={}, response_metadata={}, id='cf908f9a-3bfc-4fc8-9151-637e02b81fcb'),
  AIMessage(content='你好！有什么可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 175, 'total_tokens': 182, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '019c98fba397ca1a592faa398499ea1c', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c98fb-a220-71b0-9dfa-5e7dd13f469f-0', usage_metadata={'input_tokens': 175, 'output_tokens': 7, 'total_tokens': 182, 'input_token_details': {}, 'output_token_details': {'reasoning': 0}}),
  HumanMessage(content='你是谁啊', additional_kwargs={}, response_metadata={}, id='3a1fefe7-3c9b-4b09-a290-e3488c5c95ea'),
  AIMessage(content='我